In [ ]:
%pip install gymnasium==0.29.1
%pip install rl-zoo3==2.2.1
%pip install sb3-contrib==2.2.1
%pip install stable-baselines3==2.2.1
%pip install panda-gym==3.0.7

# %pip install torch==2.2.0
# python 310

# verbose = 0  (출력하지 않음 X)
# verbose = 1  (정보를 상세하게 출력함)
# verbose = 2  (정보를 함축적으로 출력함)

In [1]:
import panda_gym
import sys
import gymnasium
sys.modules["gym"] = gymnasium
# from stable_baselines3 import DDPG
from sb3_contrib import TQC
from stable_baselines3.common.logger import configure

# set up logger
tmp_path = "/tmp/sb3_log/"
new_logger = configure(tmp_path, ["stdout", "csv", "tensorboard"])


env = gymnasium.make("PandaPickAndPlace-v3")

model = TQC(policy="MultiInputPolicy", env=env, verbose=1)
model.set_logger(new_logger)
model.train(1000)
model.save("ppap")

Logging to /tmp/sb3_log/
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


ValueError: high <= 0

: 